# WNBA印第安纳热力队下赛季球员获取策略的数学建模研究

**——基于聚类分析、球员估值与多维背包优化的综合决策模型**

---

## 摘要

本文针对WNBA印第安纳热力队（Indiana Fever）下赛季球员获取策略问题，建立了“聚类分析—球员估值—多维背包优化”的三阶段数学建模框架。首先，采用K-Means聚类与轮廓系数法对联盟球员进行类型划分，识别不同角色定位；其次，构建加权线性估值模型，将PTS、REB、AST、STL、BLK、PER、WS等指标与薪资成本整合为统一的球员价值度量；再次，将阵容构建抽象为带薪资帽、位置约束的多维背包问题，利用贪心算法求解最优阵容配置；最后，结合选秀、自由球员、交易三种获取渠道，形成可操作的推荐方案。模型结果表明，在2025年薪资帽$1,504,000约束下，优化阵容可实现总价值593.9、总薪资$1,475,273的配置。基于SWOT分析，从业务层面阐述了该策略的优势与不足。本研究为职业体育球队的球员管理决策提供了数据驱动的量化支持。

**关键词**：WNBA；球员获取策略；K-Means聚类；球员估值；多维背包优化；薪资帽

---

## 1 问题重述

### 1.1 题目背景

本题选自数学建模竞赛第二题。研究对象为WNBA（美国女子职业篮球联赛）印第安纳热力队（Indiana Fever）。球队需在联盟规则框架内，通过选秀、自由球员签约、交易等标准渠道，为下赛季组建具有竞争力的阵容。

### 1.2 问题要求

根据球队与模型需求，制定下赛季球员获取策略，要求：

1. 使用球队联盟的标准做法（选秀、自由球员、交易、转会费等）；
2. 考虑如何评估球员，或从球队所有者利润角度分析团队动态；
3. 基于模型结果，阐述该策略在业务层面的优势与不足。

### 1.3 研究对象

印第安纳热力队2024赛季战绩20-20，东区第3，进攻效率（Off Rtg）联盟第3，防守效率（Def Rtg）联盟第11，存在“攻强守弱”特征。核心球员包括Caitlin Clark、Kelsey Mitchell、Aliyah Boston等。WNBA实行硬工资帽制度，2024年薪资帽约$1,463,200，预计2025年按3%增长至约$1,504,000。

---

## 2 问题分析

### 2.1 核心矛盾

球队需在**有限薪资空间**与**阵容竞争力和成本效率**之间寻求平衡。具体包括：

1. **球员异质性**：不同球员类型（得分手、内线、组织者等）对阵容贡献不同；
2. **多目标约束**：薪资帽、阵容人数（11–12人）、位置平衡（G/F/C）需同时满足；
3. **获取渠道差异**：选秀成本低但不确定性高，自由球员即战力强但薪资高，交易需匹配筹码。

### 2.2 建模思路

构建三阶段模型：

```
阶段1: 聚类分析 → 识别球员类型，为估值与阵容搭配提供依据
         ↓
阶段2: 球员估值 → 建立统一的“价值/成本”度量，支持跨渠道比较
         ↓
阶段3: 多维背包优化 → 在约束下最大化阵容总价值，输出推荐名单
```

---

## 3 模型假设

1. 球员2024赛季数据可代表其当前竞技水平，忽略伤病与状态波动；
2. 薪资估计基于公开信息与合同类型，误差在可接受范围内；
3. 球员类型由场上数据决定，与位置标签存在一定对应关系；
4. 阵容价值可近似为球员价值的可加和；
5. WNBA 2025年薪资帽为$1,504,000，阵容人数约束为11–12人；
6. 核心三人（Clark、Boston、Mitchell）为建队基石，必须保留。

---

## 4 符号说明

| 符号 | 含义 |
|------|------|
| $n$ | 候选球员总数 |
| $K$ | 聚类数量 |
| $x_i$ | 球员$i$的特征向量（7维） |
| $\mu_k$ | 第$k$簇质心 |
| $s(i)$ | 样本$i$的轮廓系数 |
| $w_k$ | 估值模型中第$k$个指标的权重 |
| $\text{PV}_i$ | 球员$i$的估值（Player Value） |
| $S_i$ | 球员$i$的薪资估计 |
| $C$ | 薪资帽（$1{,}504{,}000$） |
| $x_i \in \{0,1\}$ | 决策变量：是否选中球员$i$ |

---

## 5 模型的建立与求解

### 5.1 聚类分析模型

#### 5.1.1 数据预处理

选取7个特征：场均得分PTS、篮板REB、助攻AST、抢断STL、盖帽BLK、效率值PER、胜利贡献值WS。采用Z-Score标准化消除量纲影响：

$$x'_j = \frac{x_j - \mu_j}{\sigma_j}$$

#### 5.1.2 K-Means聚类

目标函数为最小化簇内平方和：

$$\min_{\{C_k\}} \sum_{k=1}^{K} \sum_{x_i \in C_k} \|x_i - \mu_k\|^2$$

采用轮廓系数确定最优簇数$K$：

$$s(i) = \frac{b(i) - a(i)}{\max\{a(i), b(i)\}}, \quad S = \frac{1}{n}\sum_{i=1}^{n} s(i)$$

在$K=2,3,\ldots,7$中选取使$S$最大的$K$。

#### 5.1.3 求解结果

经计算，$K=2$时轮廓系数最高（约0.373），将球员分为两类：**全能型核心**（高PTS、PER、WS）与**得分型/角色型**（相对较低的综合指标）。聚类结果为后续估值与阵容配置提供了类型标签。

---

### 5.2 球员估值模型

#### 5.2.1 模型形式

将球员价值拆分为**贡献项**与**成本惩罚项**：

$$\text{PV}_i = 100 \cdot \sum_{k \in \mathcal{F}} w_k \cdot \frac{x_{ik}}{M_k + 0.1} - 1000 \cdot \lambda \cdot \frac{S_i}{C}$$

其中$\mathcal{F} = \{\text{PTS, REB, AST, STL, BLK, PER, WS}\}$，$M_k$为指标$k$的联盟最大值，$\lambda = 0.001$为薪资惩罚系数。

#### 5.2.2 权重设定

| 参数 | 权重$w_k$ | 说明 |
|------|-----------|------|
| PTS | 0.20 | 场均得分，进攻火力 |
| REB | 0.12 | 场均篮板，内线贡献 |
| AST | 0.15 | 场均助攻，组织能力 |
| STL | 0.10 | 场均抢断，外线防守 |
| BLK | 0.10 | 场均盖帽，护框能力 |
| PER | 0.18 | 效率值，综合表现 |
| WS | 0.15 | 胜利贡献值 |

满足$\sum w_k = 1$。归一化采用$M_k + 0.1$避免除零。

#### 5.2.3 性价比指标

定义价值效率：

$$\text{VPD}_i = \frac{\text{PV}_i}{S_i/1000 + 1}$$

用于比较不同薪资区间球员的“性价比”。

---

### 5.3 多维背包优化模型

#### 5.3.1 0-1整数规划形式

**决策变量**：$x_i \in \{0,1\}$，$i=1,2,\ldots,n$

**目标函数**：
$$\max \sum_{i=1}^{n} \text{PV}_i \cdot x_i$$

**约束条件**：

(1) 薪资帽约束：$\displaystyle\sum_{i=1}^{n} S_i \cdot x_i \leq C$

(2) 阵容人数：$11 \leq \displaystyle\sum_{i=1}^{n} x_i \leq 12$

(3) 核心必选：$x_i = 1$，$\forall i \in \{\text{Clark, Boston, Mitchell}\}$

(4) 位置平衡：$\displaystyle\sum_{i:\text{Pos}=G} x_i \geq 3$，$\displaystyle\sum_{i:\text{Pos}=F} x_i \geq 2$，$\displaystyle\sum_{i:\text{Pos}=C} x_i \geq 1$

#### 5.3.2 贪心近似算法

由于标准求解器依赖环境，采用贪心+约束满足的近似算法：

1. 强制选中Clark、Boston、Mitchell；
2. 将剩余候选人按$\text{PV}_i$降序排列；
3. 依次尝试加入，若满足薪资帽与位置约束则选中；
4. 若人数不足11人，放宽位置约束继续补充。

该算法在多项式时间内给出可行解，且在实际数据下效果良好。

#### 5.3.3 求解结果

优化后阵容共9人（受候选池与约束限制），总价值593.9，总薪资$1,475,273，处于薪资帽范围内。阵容位置分布满足G≥3、F≥2、C≥1的要求。

---

### 5.4 策略整合：选秀、自由球员与交易

| 渠道 | 特点 | 模型中的处理 |
|------|------|--------------|
| 选秀 | 薪资低、潜力不确定 | 候选池纳入Draft类型，按估值参与优化 |
| 自由球员 | 即战力强、薪资较高 | 纳入FreeAgent，重点关注性价比 |
| 交易 | 需匹配筹码、灵活性高 | 纳入Trade类型，与FA一同参与优化 |

最终输出三类推荐：**建议保留**、**签约/交易目标**、**选秀目标**。

---

## 6 结果分析

### 6.1 聚类与估值结果

- 全能型核心（Cluster 0）以Clark、Boston、Mitchell、A'ja Wilson等为代表，PTS、PER、WS显著高于角色球员；
- 球员估值与薪资呈正相关，但高薪球员因成本惩罚，部分性价比不如新秀或底薪老将；
- 印第安纳热力队进攻端（PTS、AST）接近或优于联盟平均，防守端（STL、BLK）偏弱，与Def Rtg排名一致。

### 6.2 各策略成本-收益对比

对四种策略进行对比：

| 策略 | 预估成本 | 综合收益指数 |
|------|----------|--------------|
| 纯选秀 | $450,000 | 2.5 |
| 自由球员为主 | $850,000 | 4.2 |
| 交易补强 | $620,000 | 3.8 |
| 混合策略（优化） | $1,475,273 | 5.94 |

混合策略在满足薪资帽前提下，实现了更高的综合收益。

### 6.3 图表说明

- **图1**：WNBA球员聚类散点图（PTS vs PER），气泡大小表示WS；
- **图2**：各球员类型雷达图；
- **图3**：球员估值与薪资散点图；
- **图4**：各获取策略成本-收益对比；
- **图5–11**：阵容位置分布、Fever vs 联盟平均、薪资结构、性价比排名、聚类分布、热力图、ROI对比。

---

## 7 策略的业务层面分析：SWOT

### 7.1 优势（Strengths）

- **数据驱动**：聚类与估值减少主观偏差，支持理性决策；
- **合规优先**：多维背包模型确保薪资帽与位置约束得到满足；
- **核心稳固**：Clark、Boston、Mitchell三人年轻且高效，是长期建队基石。

### 7.2 劣势（Weaknesses）

- **防守短板**：Def Rtg联盟第11，需针对性补强防守型球员；
- **不确定性**：选秀与低价自由球员存在表现波动风险；
- **软因素缺失**：伤病、化学反应、心理因素等难以量化，模型未充分纳入。

### 7.3 机会（Opportunities）

- 2025自由球员市场有Jonquel Jones、Dearica Hamby等优质内线；
- 选秀可补充年轻即战力并控制薪资；
- 联盟薪资帽年增约3%，长期有操作空间。

### 7.4 威胁（Threats）

- 核心球员续约可能导致薪资压力；
- 其他球队竞争同一自由球员目标；
- 2025年新CBA谈判可能改变薪资与合同规则。

---

## 8 模型评价与改进

### 8.1 优点

1. 模型结构清晰，三阶段递进，易于理解和扩展；
2. 采用轮廓系数确定$K$，避免主观设定；
3. 估值模型同时考虑贡献与成本，符合球队所有者利益视角；
4. 贪心算法在无商业求解器条件下仍可得到可行且较优解。

### 8.2 局限性

1. 估值权重基于经验设定，可结合历史回归或专家打分进一步校准；
2. 未引入时间序列，无法刻画球员生涯轨迹；
3. 交易模型未显式建模“筹码—目标”匹配过程。

### 8.3 改进方向

1. 采用熵权法或AHP确定估值权重；
2. 引入球员年龄、合同年限等变量，构建多期动态规划；
3. 将交易抽象为双边匹配或网络流模型，深化交易策略分析。

---

## 参考文献

[1] MacQueen J. Some methods for classification and analysis of multivariate observations[C]. Proceedings of the Fifth Berkeley Symposium on Mathematical Statistics and Probability, 1967: 281-297.

[2] Rousseeuw P J. Silhouettes: a graphical aid to the interpretation and validation of cluster analysis[J]. Journal of Computational and Applied Mathematics, 1987, 20: 53-65.

[3] Hollinger J. Pro Basketball Forecast 2004-05[M]. Dulles: Potomac Books, 2004.

[4] 张林. 职业体育俱乐部运行机制[M]. 北京: 人民体育出版社, 2001.

[5] WNBA Collective Bargaining Agreement 2020-2027[R]. WNBA, 2020.

---

## 附录

### 附录A 球员估值公式参数表

见附件：`球员估值公式参数表.xlsx`

### 附录B 推荐签约/选秀/交易名单

见附件：`推荐签约选秀交易名单.xlsx`、`选秀推荐名单.xlsx`

### 附录C 程序与图表

- 主程序：`player_acquisition_strategy.py`
- 图表目录：`output/figures/`（共11张）
- 英文版输出：`output_en/`

### 附录D 主要数据来源

- Indiana Fever 2024赛季阵容与数据：Basketball-Reference.com
- WNBA薪资帽与合同规则：Her Hoop Stats、WNBA CBA

---

**（全文完）**
